In [1]:
import glob
import re
import random
import random
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm.notebook import trange, tqdm
from deep_shadow import *
from utils import *

%load_ext autoreload
%autoreload 2

In [2]:
deep_shadow = DeepShadow(512, 512)
deep_shadow.restore('training_checkpoints/evaluation_new/uniform_cities')

In [3]:
height_path = 'data/heights_new/'
output_folder = 'data/shadows_new/'

In [5]:
paths = glob.glob('%s/*/'%height_path)
for path in tqdm(paths):
    files = glob.glob(path+'/*/*/*.png')
    for filepath in tqdm(files, leave=False):
        dates = ['spring', 'summer', 'winter']
        for date in dates:
            tks = os.path.normpath(filepath).split(os.path.sep)
            city, zoom, i, j = tks[-4], int(tks[-3]), int(tks[-2]), int(tks[-1][:-4])
            
            folder = '%s/%s-%s/%d/%d/'%(output_folder,city,date,zoom,i)
            outputpath = '%s/%s-%s/%d/%d/%d.png'%(output_folder,city,date,zoom,i,j)
            
            # Skip if file exists
            if os.path.exists(outputpath):
                continue
            
            if not os.path.exists(folder):
                os.makedirs(folder)
            
            input_height, input_lat, input_date = load_input_grid(height_path, city, date, zoom, i, j)
            input_height, input_lat, input_date = normalize_input(input_height, input_lat, input_date)
            input_height = np.array(input_height).reshape(1,512,512,1)
            input_lat = np.array(input_lat).reshape(1,512,512,1)
            input_date = np.array(input_date).reshape(1,512,512,1)
            prediction = deep_shadow.generator([input_height, input_lat, input_date], training=True)
            
            height = input_height[:,128:-128,128:-128,:]
            height = height * 0.5 + 0.5
            prediction = prediction.numpy()[:,128:-128,128:-128,:]
            prediction = prediction * 0.5 + 0.5
            
            cv2.imwrite(outputpath, prediction[0]*255)
            
#             if random.random() <= 0.01:
#                 plot_result(height[0], prediction[0])
        
#             plot_result(height[0],  prediction[0])
#             target_image = tf.io.read_file('data/shadows/%s-%s/%d/%d/%d.png'%(city,date,zoom,i,j))
#             target_image = tf.io.decode_png(target_image)[:,:,0]
#             target_image = tf.reshape(target_image, (256, 256, 1))
#             target_image = tf.cast(target_image, tf.float32)
#             plot_comparison(height[0], target_image, prediction[0])
#             break
#         break
#     break

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/134 [00:00<?, ?it/s]